# 인스타그램 해시태그 크롤링
## 필요한 라이브러리 import

In [14]:
#%pip install selenium
#%pip install webdriver_manager
#%pip install lxml
#%pip install openpyxl
#%pip install --upgrade pandas
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from urllib.request import urlretrieve
from urllib.parse import quote_plus
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from lxml import html
import urllib.request
import requests
import os
import time
import re
import random

## 해시태그 url 변수 저장

In [2]:
# 해시태그 검색하여 url 변수에 저장
def Search_hashtag(word):
    url = "https://www.instagram.com/explore/tags/" + str(word)
    return url

## 열린 페이지에서 첫 번째 게시물 클릭

In [3]:
def select_first(driver):
  try:
    first_post = WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.CSS_SELECTOR, 'a[href^="/p/"]'))
    )
    first_post.click()
  except Exception as e:
    print("첫 번째 게시물 선택 오류: {e}")
    
  #first = driver.find_element(By.CSS_SELECTOR, "div._aagw")
  #time.sleep(3)

## 첫 게시물 클릭 후 다음 게시물 클릭

In [4]:
def move_next_post(driver):
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "._abl-"))
        )
        next_button.click()
    except Exception as e:
        print(f"다음 게시물로 이동 오류: {e}")
    # right = driver.find_element(By.CSS_SELECTOR, "div._aaqg._aaqh")
    # time.sleep(3)

## 크롤링 함수

In [ ]:
# 이미지 저장
def img_save(link, index):
    try:
        wait = WebDriverWait(driver, 10)
        img_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3')))
    
        # 첫 번째 이미지 요소만 저장
        if len(img_elements) > 0:
            img_element = img_elements[index]
            img_url = img_element.get_attribute('src')
        
            # img_url 값 확인
            print(f"이미지 URL: {img_url}")
        
        return img_url
    except Exception as e:
        print(f"이미지 저장 오류: {e}")
    return None

# 함수 정의: 이미지, 링크 가져오기
def collect_links(driver, index):
    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        link = soup.select('a[href^="/p/"]')
        return ['https://www.instagram.com' + link['href'] for link in link[:index]]
    except Exception as e:
        print(f"링크 수집 오류: {e}")
        return []

def get_content(driver, index):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    # 링크 수집
    link = collect_links(driver, index)
    print("수집된 링크: ")
    print(link)
    
    if link:
        # 이미지 저장
        img_url = img_save(link, index)
        data = {"img_url": img_url, "link": link}
    return data

## 로그인 함수

In [6]:

def login_instagram(driver, username, password):
    try:
        username_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input[name='username']"))
        )
        username_input.send_keys(username)

        password_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input[name='password']"))
        )
        password_input.send_keys(password)

        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))
        )
        login_button.click()
        return True
    except Exception as e:
        print(f"로그인 오류: {e}")
        return False


## 크롤링 시작

In [51]:
# 크롬 브라우저 열기
try:
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)
    driver.implicitly_wait(10)
    driver.get('https://www.instagram.com')
except Exception as e:
    print(f"웹드라이버 초기화 오류: {e}")
    raise

time.sleep(3)

#인스타그램 로그인
username = 'semlng'
password = 'wkdtpals2wmrkt!'
if not login_instagram(driver, username, password):
    driver.quit()
    raise Exception("로그인 실패")

time.sleep(5)

# 해시태그 검색
word = input("검색할 해시태그를 입력하세요: ")
url = Search_hashtag(word)

# 검색 결과 페이지 열기
print(f"해시태그 {word}에 대한 검색 결과 페이지 열기")
driver.get(url)
time.sleep(10)

# 첫 번째 게시물 클릭
select_first(driver)

# 데이터 수집 시작
results = []
# 수집할 게시물의 수
target = 3
for i in range(target):
    print(f"{i+1}/{target} 번째 게시물 수집 중...")
    try:
        data = get_content(driver, i)
        results.append(data)
        move_next_post(driver)
    except Exception as e:
        print(f"게시물 수집 오류 : {e}")
        move_next_post(driver)
    time.sleep(5)
    
print(results)

# 결과를 데이터프레임으로 저장
date = datetime.today().strftime('%Y-%m-%d')
results_df = pd.DataFrame(results)

# 엑셀 파일로 저장
results_df.to_excel('instagram_crawling.xlsx')

# WebDriver 종료
driver.quit()

해시태그 마뗑킴코디에 대한 검색 결과 페이지 열기
1/3 번째 게시물 수집 중...
수집된 링크: 
https://www.instagram.com/p/C7OyNSAPfJB/?img_index=1
이미지 URL: https://scontent-ssn1-1.cdninstagram.com/v/t51.29350-15/445117187_1437462806888811_7791906540257023493_n.jpg?stp=dst-jpg_e35&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi4xNDQweDE4MDAuc2RyLmYyOTM1MCJ9&_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=103&_nc_ohc=1oRpqWfLL0gQ7kNvgH7IfEK&edm=ALQROFkBAAAA&ccb=7-5&ig_cache_key=MzM3Mjg1Mzk3MzUzMzc0MDA3Ng%3D%3D.2-ccb7-5&oh=00_AYBTO3XTk-koRKx9Zfx0C2JCGQ9Tw76AX-Vz5wJoj7scow&oe=666F510F&_nc_sid=fc8dfb
2/3 번째 게시물 수집 중...
수집된 링크: 
https://www.instagram.com/p/CiO2v1zpLcI/
이미지 URL: https://scontent-ssn1-1.cdninstagram.com/v/t51.29350-15/306043499_2803194229824615_6591241000885270086_n.jpg?stp=dst-jpg_e35&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi4xNDQweDE0NDAuc2RyLmYyOTM1MCJ9&_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=103&_nc_ohc=fjATgC4BSoAQ7kNvgE7PKhK&edm=ALQROFkBAAAA&ccb=7-5&ig_cache_key=MjkyMjUxMzk5MDMxMTc4NjI0OA%3D%3D.2-ccb7-5&

In [ ]:
# 변환할 엑셀 파일 경로
excel_file = r'C:\Users\wkdwoo\Desktop\장세민\이력서\포트폴리오\Git\패션브랜드 소비자 대시보드 제작 프로젝트\크롤링\마뗑킴\#merged_Martinkim.xlsx'

# CSV 파일로 저장할 경로
csv_file = r'C:\Users\wkdwoo\Desktop\장세민\이력서\포트폴리오\Git\패션브랜드 소비자 대시보드 제작 프로젝트\크롤링\마뗑킴\#merged_Martinkim.csv'

# 엑셀 파일 읽기
df = pd.read_excel(excel_file)

# CSV 파일로 저장
df.to_csv(csv_file, index=False)